In [ ]:
from bs4 import BeautifulSoup
import re
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import unicodedata

path_input = Path("/home/agnese.lombardi/progetto/ToM_concordia/data/")
path_output = Path("/home/agnese.lombardi/progetto/ToM_concordia/analysis")
scenario_list = ["1", "2", "3", "4", "5"]
index_list = ["0", "1", "2", "3", "4", "5", "6", "7"]
phenomenas = ["IR", "IS","ID", "IR_Os", "IT", "VI_IH", "VI_IQ", "VI_IS"]

def generate_path(scenario, path_input, path_output, phen):
    input_list = [path_input/f'{phen}_task{scenario}.csv']
    output_list = [path_output/f'{phen}_Task{scenario}.csv']
    return input_list, output_list

def normalize_string(s):
    return unicodedata.normalize('NFKD', s.strip())


for scenario in scenario_list:
 for phen in phenomenas:
   input_list, output_list = generate_path(scenario, path_input, path_output, phen)
   for input, output in zip(input_list, output_list):
      df = pd.read_csv(input)
      df["Model_lc_action"] = ""
      df["Model_ilc_action"] = ""
      for i, row in tqdm(df.iterrows()):
         locutor = normalize_string(row.LcName)
         illocutor = normalize_string(row.IlcName)   
           
         for n in index_list:
          for phenomena in phenomenas:
            with open(f'/home/agnese.lombardi/progetto/ToM_concordia/queries/{phenomena}_Task{scenario}_{n}.html', 'r', encoding='utf-8') as file:
               html_content = file.read()
               soup = BeautifulSoup(html_content, 'html.parser')
               game_master = soup.find(id = "Game Master", class_ = "tabcontent").text
            
               pattern = r'Name(.*?)Action attempt(.*?)Context for action selection and components'
               pattern_keffect = r'Known effect(.*?)Unknown effect'
               pattern_ueffect = r'Unknown effect(.*?)Chain of thought'
               matches = re.findall(pattern, game_master, re.DOTALL)
               matches_keffect = re.findall(pattern_keffect, game_master, re.DOTALL)
               matches_ueffect = re.findall(pattern_ueffect, game_master, re.DOTALL)
            
               if matches:
                  first_player = normalize_string(matches[0][0].strip()) if len(matches) > 0 else ""
                  second_player = normalize_string(matches[1][0].strip()) if len(matches) > 1 else ""
                  print("First player:" + first_player)
                  print("Second player:" + second_player)

                  first_action_attempt = matches[0][1].strip() if len(matches) > 0 else ""
                  second_action_attempt = matches[1][1].strip() if len(matches) > 1 else ""
                  first_keffect = matches_keffect[0].strip() if len(matches_keffect) > 0 else ""
                  second_keffect = matches_keffect[1].strip() if len(matches_keffect) > 1 else ""
                  first_ueffect = matches_ueffect[0].strip() if len(matches_ueffect) > 0 else ""
                  second_ueffect = matches_ueffect[1].strip() if len(matches_ueffect) > 1 else ""
                  print("Effect:" + first_keffect)
                  print("Effect" + second_keffect)
                  
                  if first_player in locutor:
                     df.loc[i, "Model_lc_action"] = first_action_attempt
                     df.loc[i, "known_effect_lc"] = first_keffect
                     df.loc[i, "unknown_effect_lc"] = first_ueffect
                  elif first_player in illocutor:
                     df.loc[i, "Model_ilc_action"] = first_action_attempt
                     df.loc[i, "known_effect_ilc"] = first_ueffect
                     df.loc[i, "unknown_effect_ilc"] = first_ueffect
                  elif second_player in locutor:
                     df.loc[i, "Model_lc_action"] = second_action_attempt
                     df.loc[i, "known_effect_lc"] = second_keffect
                     df.loc[i, "unknown_effect_lc"] = second_ueffect
                  elif second_player in illocutor:
                     df.loc[i, "Model_ilc_action"] = second_action_attempt
                     df.loc[i, "known_effect_ilc"] = second_keffect
                     df.loc[i, "unknown_effect_ilc"] = second_ueffect
                  df['Lc_knw'] = df.apply(lambda row: str(row['Locutor_answer']) in str(row['Model_lc_action']), axis=1)
                  df['Ilc_knw'] = df.apply(lambda row: str(row['Illocutor_answer']) in str(row['Model_ilc_action']), axis=1)
                  
                  df.to_csv(output, index= False)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

path_output = Path("/home/agnese.lombardi/progetto/ToM_concordia/analysis")

percent_data = []

TASK = {
    "IR": "Indirect Requests",
    "IS": "Indirect Suggestions",
    "IR_Os": "Indirect Requests (Old stimuli)",
    "IT" : "Indirect Threats",
    "ID" : "Indirect Declinations",
    "VI_IH" : "Indirect Hyperboles",
    "VI_IS" : "Indirect Sarcasms",
    "VI_IQ" : "Indirect Questions"
}

# Gather data for each scenario and phenomenon
for scenario in scenario_list:
    for phen in phenomenas:
        output_path = path_output / f'{phen}_Task{scenario}.csv'
        if output_path.exists():  # Ensure the file exists
            df = pd.read_csv(output_path)

            # Calculate percentage of True values in Lc_knw and Ilc_knw columns
            lc_knw_counts = df["Lc_knw"].value_counts(normalize=True) * 100
            ilc_knw_counts = df["Ilc_knw"].value_counts(normalize=True) * 100

            percent_data.append({
                'Scenario': scenario,
                'Type': 'Listener',
                'Phenomena': phen,
                'Percentage': ilc_knw_counts.get(True, 0)
            })
        else:
            print(f"File {output_path} not found.") 

# Convert percentage data into a DataFrame
percent_df = pd.DataFrame(percent_data)
if percent_df.empty:
    print("No data to plot. Check input files.")
else:
    y_max = percent_df['Percentage'].max() + 10
    num_phenomena = len(phenomenas)
    rows = (num_phenomena + 1) // 2 
    fig, axes = plt.subplots(rows, 2, figsize=(16, rows * 4))
    axes = axes.flatten() 
    for idx, phen in enumerate(phenomenas):
        ax = axes[idx]
        data = percent_df[percent_df['Phenomena'] == phen]

        if not data.empty: 
            sns.barplot(
                x='Scenario', 
                y='Percentage', 
                hue='Type', 
                data=data,
                palette={'Listener': 'orange'},
                ax=ax
            )
            ax.set_title(f'Percentage of Correct Answers for {phen} Tasks')
            ax.set_ylabel('Percentage (%)')
            ax.set_xlabel('Task Number')
            ax.set_ylim(0, y_max)
            ax.legend(title='Type')
        else:
            ax.set_visible(False)  

    for idx in range(len(phenomenas), len(axes)):
        fig.delaxes(axes[idx])

    plt.tight_layout()
    plt.show()


In [ ]:
# Create DataFrame to store ratings for heatmap
heatmap_data = pd.DataFrame(index=["Speaker", "Listener"], columns=scenario_list)
zero_counts = {"Speaker": 0, "Listener": 0}  # Track zeros for percentage calculation
total_counts = {"Speaker": 0, "Listener": 0}  # Track total values per role

for scenario in scenario_list:
    ratings_speaker = []
    ratings_listener = []

    for phen in phenomenas:
        output_path = path_output / f'{phen}_Task{scenario}_ratings.csv'
        
        if output_path.exists():
            df = pd.read_csv(output_path)

            # Ensure columns exist
            if "Rating_lc" in df.columns and "Rating_ilc" in df.columns:
                df["Rating_lc"] = pd.to_numeric(df["Rating_lc"], errors="coerce")
                df["Rating_ilc"] = pd.to_numeric(df["Rating_ilc"], errors="coerce")

                # Collect ratings and count zeros
                valid_speaker_ratings = df["Rating_lc"][df["Rating_lc"] != 0].dropna().tolist()
                valid_listener_ratings = df["Rating_ilc"][df["Rating_ilc"] != 0].dropna().tolist()

                ratings_speaker.extend(valid_speaker_ratings)
                ratings_listener.extend(valid_listener_ratings)

                # Update zero counts
                zero_counts["Speaker"] += (df["Rating_lc"] == 0).sum()
                zero_counts["Listener"] += (df["Rating_ilc"] == 0).sum()
                total_counts["Speaker"] += df["Rating_lc"].count()
                total_counts["Listener"] += df["Rating_ilc"].count()

    # Compute the mean rating for each scenario, ignoring 0s
    heatmap_data.at["Speaker", scenario] = sum(ratings_speaker) / len(ratings_speaker) if ratings_speaker else None
    heatmap_data.at["Listener", scenario] = sum(ratings_listener) / len(ratings_listener) if ratings_listener else None

# Convert to numeric format
heatmap_data = heatmap_data.astype(float)

# Compute zero percentages
zero_percentage_speaker = (zero_counts["Speaker"] / total_counts["Speaker"] * 100) if total_counts["Speaker"] else 0
zero_percentage_listener = (zero_counts["Listener"] / total_counts["Listener"] * 100) if total_counts["Listener"] else 0

# Plot the heatmap
plt.figure(figsize=(8, 5))
sns.heatmap(heatmap_data, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)

# Display zero percentages in a separate box (Top-Right Corner with More Space)
text_str = f"Zero %\nSpeaker: {zero_percentage_speaker:.1f}%\nListener: {zero_percentage_listener:.1f}%"
plt.gcf().text(0.92, 0.85, text_str, fontsize=10, 
               bbox=dict(facecolor='white', alpha=0.7, boxstyle="round,pad=0.4"))  # Increased padding

plt.show()
